<a href="https://colab.research.google.com/github/novalsunn123/finetune/blob/main/qwen-parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B",
    max_seq_length = 2048,   # Context length - can be longer, but uses more memory
    load_in_4bit = True,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = False, # We have full finetuning now!
    # token = "hf_...",      # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.12: Fast Qwen3 patching. Transformers: 4.53.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,  # Best to choose alpha = rank or rank*2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,   # We support rank stabilized LoRA
    loftq_config = None,  # And LoftQ
)

Unsloth 2025.6.12 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [ ]:
# 3. Hàm xem dataset
def view_dataset(dataset, name="Dataset", num_samples=2):
    print(f"\n--- Xem {name} ---")
    print(f"Số bản ghi: {len(dataset)}")
    print(f"Các cột: {dataset.column_names}")
    print(f"Mẫu {num_samples} bản ghi đầu tiên:")
    for i in range(min(num_samples, len(dataset))):
        print(f"Bản ghi {i}:")
        print(dataset[i])
    print("-------------------")

In [ ]:
import json
import os
from datasets import Dataset
from google.colab import files  # Chỉ cần nếu dùng Google Colab

print("\nTải file security_dataset.json...")
dataset_path = '/content/dataset.json'

# Kiểm tra file tồn tại, nếu không thì yêu cầu tải lên
if not os.path.exists(dataset_path):
    print(f"File '{dataset_path}' không tồn tại. Tải file lên...")
    uploaded = files.upload()
    if 'dataset.json' not in uploaded:
        print("Error: Vui lòng tải file 'security_dataset.json'")
        raise FileNotFoundError("File 'security_dataset.json' không được tải lên.")

try:
    with open(dataset_path, 'r') as f:
        data = json.load(f)

    # Chuyển đổi dữ liệu thành danh sách các từ điển
    flattened_data = [
        {
            "input": pair[0]["input"],
            "content": pair[1]["content"],
            "input_role": pair[0]["role"],
            "content_role": pair[1]["role"]
        }
        for pair in data
    ]

    # Chuyển thành Dataset để kiểm tra
    raw_dataset = Dataset.from_list(flattened_data)
    print(raw_dataset)  # Hiển thị thông tin dataset để kiểm tra

except FileNotFoundError:
    print(f"Error: File '{dataset_path}' not found.")
    raise
except json.JSONDecodeError:
    print("Error: Invalid JSON format in the file.")
    raise
except Exception as e:
    print(f"Error: {str(e)}")
    raise


Tải file security_dataset.json...
Dataset({
    features: ['input', 'content', 'input_role', 'content_role'],
    num_rows: 62
})


dataset phân tích log openappsec

In [ ]:
def generate_conversation(examples):
    conversations = []
    for input_data, content_data in zip(examples["input"], examples["content"]):
        conversation = [
            {
                "role": "user",
                "content": f"{input_data}\ndo log processing for me from above file into understandable form for machine learning to read"
            },
            {
                "role": "assistant",
                "content": f"{content_data}"
            }
        ]
        conversations.append(conversation)
    return {"conversations": conversations}

print("\nTạo conversation...")
conversation_dataset = raw_dataset.map(generate_conversation, batched=True)
view_dataset(conversation_dataset, "Conversation Dataset")


Tạo conversation...


Map:   0%|          | 0/62 [00:00<?, ? examples/s]


--- Xem Conversation Dataset ---
Số bản ghi: 62
Các cột: ['input', 'content', 'input_role', 'content_role', 'conversations']
Mẫu 2 bản ghi đầu tiên:
Bản ghi 0:
{'input': '{"eventTime": "2025-06-20T14:04:40.273", "eventName": "Web AppSec Policy Loaded Successfully", "eventSeverity": "Info", "eventPriority": "Low", "eventType": "Event Driven", "eventLevel": "Log", "eventLogLevel": "info", "eventAudience": "Security", "eventAudienceTeam": "", "eventFrequency": 0, "eventTags": ["Threat Prevention"], "eventSource": {"agentId": "fbfb6a19-daf7-41e5-a25c-bf3a1472e97a", "eventTraceId": "", "eventSpanId": "", "issuingEngineVersion": "", "serviceName": "HTTP Transaction Handler", "serviceId": "1"}, "eventData": {"logIndex": 1}}', 'content': '{"eventTime": "2025-06-20T14:04:40.273", "eventName": "Web AppSec Policy Loaded Successfully", "eventSeverity": "Info", "eventPriority": "Low", "eventType": "Event Driven", "eventLevel": "Log", "eventLogLevel": "info", "eventAudience": "Security", "eventAudi

In [ ]:
# Kiểm tra reasoning_conversations
print("\nKiểm tra reasoning_conversations:")
for i in range(min(2, len(conversation_dataset))):
    print(f"Chuỗi {i}:")
    print(conversation_dataset[i])


Kiểm tra reasoning_conversations:
Chuỗi 0:
{'input': '{"eventTime": "2025-06-20T14:04:40.273", "eventName": "Web AppSec Policy Loaded Successfully", "eventSeverity": "Info", "eventPriority": "Low", "eventType": "Event Driven", "eventLevel": "Log", "eventLogLevel": "info", "eventAudience": "Security", "eventAudienceTeam": "", "eventFrequency": 0, "eventTags": ["Threat Prevention"], "eventSource": {"agentId": "fbfb6a19-daf7-41e5-a25c-bf3a1472e97a", "eventTraceId": "", "eventSpanId": "", "issuingEngineVersion": "", "serviceName": "HTTP Transaction Handler", "serviceId": "1"}, "eventData": {"logIndex": 1}}', 'content': '{"eventTime": "2025-06-20T14:04:40.273", "eventName": "Web AppSec Policy Loaded Successfully", "eventSeverity": "Info", "eventPriority": "Low", "eventType": "Event Driven", "eventLevel": "Log", "eventLogLevel": "info", "eventAudience": "Security", "eventAudienceTeam": "", "eventFrequency": 0, "eventTags_0": "Threat Prevention", "eventSource_agentId": "fbfb6a19-daf7-41e5-a2

In [ ]:
import pandas as pd
from datasets import Dataset
# 8. Chuyển thành Dataset cuối cùng
print("\nTạo Dataset cuối cùng...")
data_series = pd.Series(conversation_dataset)
data_series.name = "text"
combined_dataset = Dataset.from_pandas(pd.DataFrame(data_series))
combined_dataset = combined_dataset.shuffle(seed=3407)
view_dataset(combined_dataset, "Final Dataset")


Tạo Dataset cuối cùng...

--- Xem Final Dataset ---
Số bản ghi: 62
Các cột: ['text']
Mẫu 2 bản ghi đầu tiên:
Bản ghi 0:
{'text': {'content': '{"eventTime": "2025-06-21T15:06:06.956", "eventName": "Web Request", "eventSeverity": "Medium", "eventPriority": "Medium", "eventType": "Event Driven", "eventLevel": "Log", "eventLogLevel": "info", "eventAudience": "Security", "eventAudienceTeam": "", "eventFrequency": 0, "eventTags_0": "Threat Prevention", "eventTags_1": "Web Application & API Protection", "eventSource_agentId": "Unknown", "eventSource_eventTraceId": "", "eventSource_eventSpanId": "", "eventSource_issuingEngineVersion": "1.1.26-open-source", "eventSource_serviceName": "HTTP Transaction Handler", "eventSource_serviceId": "1", "eventSource_assetId": "Any", "eventSource_assetName": "Any", "eventData_logIndex": 8, "eventData_eventReferenceId": "baa50603-e006-43d5-861f-facdcdb6e01e", "eventData_assetId": "Any", "eventData_assetName": "Any", "eventData_eventConfidence": "Medium", "ev

In [ ]:
import pandas as pd
from datasets import Dataset
import json


print("\nKiểm tra reasoning_conversations:")
for i in range(min(2, len(conversation_dataset))):
    print(f"Chuỗi {i}:")
    print(conversation_dataset[i])

def fill_none_values(obj, default_value=""):
    if isinstance(obj, dict):
        return {k: fill_none_values(v, default_value) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [fill_none_values(elem, default_value) for elem in obj]
    elif obj is None or (isinstance(obj, str) and obj.lower() == "null"):
        return default_value
    else:
        return obj

# Hàm định dạng conversation thành chuỗi văn bản
def format_conversation(example):
    processed_conversations = [] # Đổi tên biến để rõ ràng hơn

    for turn in example["conversations"]:
        try:
            # Tải nội dung JSON hiện tại của lượt đối thoại
            # Cả user và assistant đều được xử lý ở đây
            current_content_dict = json.loads(turn["content"])

            # Áp dụng hàm điền giá trị mặc định cho nội dung đã tải
            cleaned_content_dict = fill_none_values(current_content_dict, default_value="")

            # Ghi lại nội dung đã làm sạch dưới dạng chuỗi JSON
            turn["content"] = json.dumps(cleaned_content_dict, ensure_ascii=False)
        except json.JSONDecodeError as e:
            # Nếu content không phải JSON hợp lệ, in lỗi và giữ nguyên content.
            # Bạn có thể cân nhắc gán turn["content"] = default_value (ví dụ: "") nếu muốn làm sạch cả chuỗi không phải JSON.
            # print(f"JSONDecodeError for role '{turn['role']}': {turn['content']}. Error: {e}")
            pass # Giữ nguyên content nếu không thể phân tích JSON

        processed_conversations.append(turn)

    text = json.dumps(processed_conversations, ensure_ascii=False)
    return {"text": text}

# print("\nTạo conversation...")
# conversation_dataset = raw_dataset.map(generate_conversation, batched=True)
# view_dataset(conversation_dataset, "Conversation Dataset")

print("\nTạo Dataset cuối cùng...")
# Áp dụng hàm format_conversation để tạo cột text
combined_dataset = conversation_dataset.map(format_conversation, batched=False)

# Xóa các cột không cần thiết, chỉ giữ cột text
combined_dataset = combined_dataset.remove_columns([col for col in combined_dataset.column_names if col != "text"])

# Xáo trộn dataset
combined_dataset = combined_dataset.shuffle(seed=3407)

# Hiển thị dataset
def view_dataset(dataset, name):
    print(f"\n--- Xem {name} ---")
    print(f"Số bản ghi: {len(dataset)}")
    print(f"Các cột: {dataset.column_names}")
    print("Mẫu 2 bản ghi đầu tiên:")
    for i in range(min(60, len(dataset))):
        print(f"Bản ghi {i}:")
        print(dataset[i])

view_dataset(combined_dataset, "Final Dataset")


Kiểm tra reasoning_conversations:
Chuỗi 0:
{'input': '{"eventTime": "2025-06-20T14:04:40.273", "eventName": "Web AppSec Policy Loaded Successfully", "eventSeverity": "Info", "eventPriority": "Low", "eventType": "Event Driven", "eventLevel": "Log", "eventLogLevel": "info", "eventAudience": "Security", "eventAudienceTeam": "", "eventFrequency": 0, "eventTags": ["Threat Prevention"], "eventSource": {"agentId": "fbfb6a19-daf7-41e5-a25c-bf3a1472e97a", "eventTraceId": "", "eventSpanId": "", "issuingEngineVersion": "", "serviceName": "HTTP Transaction Handler", "serviceId": "1"}, "eventData": {"logIndex": 1}}', 'content': '{"eventTime": "2025-06-20T14:04:40.273", "eventName": "Web AppSec Policy Loaded Successfully", "eventSeverity": "Info", "eventPriority": "Low", "eventType": "Event Driven", "eventLevel": "Log", "eventLogLevel": "info", "eventAudience": "Security", "eventAudienceTeam": "", "eventFrequency": 0, "eventTags_0": "Threat Prevention", "eventSource_agentId": "fbfb6a19-daf7-41e5-a2

Map:   0%|          | 0/62 [00:00<?, ? examples/s]


--- Xem Final Dataset ---
Số bản ghi: 62
Các cột: ['text']
Mẫu 2 bản ghi đầu tiên:
Bản ghi 0:
{'text': '[{"content": "{\\"eventTime\\": \\"2025-06-21T15:06:06.956\\", \\"eventName\\": \\"Web Request\\", \\"eventSeverity\\": \\"Medium\\", \\"eventPriority\\": \\"Medium\\", \\"eventType\\": \\"Event Driven\\", \\"eventLevel\\": \\"Log\\", \\"eventLogLevel\\": \\"info\\", \\"eventAudience\\": \\"Security\\", \\"eventAudienceTeam\\": \\"\\", \\"eventFrequency\\": 0, \\"eventTags\\": [\\"Threat Prevention\\", \\"Web Application & API Protection\\"], \\"eventSource\\": {\\"agentId\\": \\"Unknown\\", \\"eventTraceId\\": \\"\\", \\"eventSpanId\\": \\"\\", \\"issuingEngineVersion\\": \\"1.1.26-open-source\\", \\"serviceName\\": \\"HTTP Transaction Handler\\", \\"serviceId\\": \\"1\\", \\"assetId\\": \\"Any\\", \\"assetName\\": \\"Any\\"}, \\"eventData\\": {\\"logIndex\\": 8, \\"eventReferenceId\\": \\"baa50603-e006-43d5-861f-facdcdb6e01e\\", \\"assetId\\": \\"Any\\", \\"assetName\\": \\"Any\\"

In [ ]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model=model,  # Đảm bảo model đã được tải (ví dụ: Qwen3-4B)
    tokenizer=tokenizer,  # Đảm bảo tokenizer đã được tải
    train_dataset=combined_dataset,  # Dataset đã chuẩn bị trước đó
    eval_dataset=None,
    args=SFTConfig(
        dataset_text_field="text",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=30,
        learning_rate=2e-4,
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        report_to="none",
        output_dir="/content/qwen3-4b-finetuned-security",  # Thư mục lưu mô hình sau huấn luyện
        save_strategy="steps",  # Lưu checkpoint sau mỗi số bước
        save_steps=10,  # Lưu sau mỗi 10 bước (vì max_steps=30, sẽ có 3 checkpoint)
    ),
)

# Bắt đầu huấn luyện
trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/62 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 62 | Num Epochs = 4 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 66,060,288 of 4,000,000,000 (1.65% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.192900
2,1.240100
3,1.213800
4,1.096800
5,1.031700
6,1.065100
7,0.949300
8,0.941800
9,0.863000
10,0.786600


TrainOutput(global_step=30, training_loss=0.5932672033707301, metrics={'train_runtime': 702.6104, 'train_samples_per_second': 0.342, 'train_steps_per_second': 0.043, 'total_flos': 7956403612956672.0, 'train_loss': 0.5932672033707301})

In [ ]:
from transformers import TextStreamer
print("\nKiểm tra mô hình với một log sự kiện...")
messages = [
    {
        "role": "user",
        "content": """{"eventTime": "2025-06-20T14:41:15.313","eventName": "Web Request","eventSeverity": "Critical","eventPriority": "High","eventType": "Event Driven","eventLevel": "Log","eventLogLevel": "info","eventAudience": "Security","eventAudienceTeam": "","eventFrequency": 0,"eventTags": ["Threat Prevention","Web Application & API Protection"],"eventSource": {"agentId": "fbfb6a19-daf7-41e5-a25c-bf3a1472e97a","eventTraceId": "","eventSpanId": "","issuingEngineVersion": "1.1.26-open-source","serviceName": "HTTP Transaction Handler","serviceId": "1","assetId": "Any","assetName": "Any"},"eventData": {"logIndex": 9,"eventReferenceId": "ca55b172-8ec9-4e32-a094-aa555da0b187","assetId": "Any","assetName": "Any","eventConfidence": "Very High","sourceIP": "192.168.136.1","httpSourceId": "192.168.136.1","sourcePort": 50911,"httpHostName": "192.168.136.128","httpMethod": "POST","assetId": "Any","assetName": "Any","ruleId": "Any","securityAction": "Detect","waapOverride": "None","practiceType": "Threat Prevention","practiceSubType": "Web Application","ruleName": "Any","practiceId": "3943d645-b4dd-4185-b9e8-bbf46642b795","practiceName": "local_policy/appsec-best-practice","waapIncidentType": "SQL Injection","matchedSample": "\"' or '1'='1' --\",","matchedLocation": "body","matchedParameter": "username","waapFoundIndicators": "[\", ', '=', --, =, or, probing, regex_postfix_0, regex_postfix_1, regex_prefix_0, regex_prefix_1, regex_sqli_0, regex_sqli_13, regex_sqli_15, regex_sqli_17, regex_sqli_22, regex_sqli_26]","matchedIndicators": "[\", ', '=', --, =, or, probing, regex_postfix_0, regex_postfix_1, regex_prefix_0, regex_prefix_1, regex_sqli_0, regex_sqli_13, regex_sqli_15, regex_sqli_17, regex_sqli_22, regex_sqli_26]","learnedIndicators": "","waapUserReputationScore": 124,"waapUserReputation": "Low","waapUriFalsePositiveScore": 864,"waapKeywordsScore": 971,"reservedNgenA": 0,"waapFinalScore": 1000,"waapCalculatedThreatLevel": 4}}
do log processing for me from above file into understandable form for machine learning to read Fields that are left blank are replaced with the value "null" just return me json file, nothing else """
    }
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False,
)

print("Kết quả:")
_ = model.generate(
    **tokenizer(text, return_tensors="pt").to("cuda"),
    max_new_tokens=1000,
    temperature=0.7,
    top_p=0.8,
    top_k=20,
    streamer=TextStreamer(tokenizer, skip_prompt=True),
)


Kiểm tra mô hình với một log sự kiện...
Kết quả:
{
  "eventTime": "2025-06-20T14:41:15.313",
  "eventName": "Web Request",
  "eventSeverity": "Critical",
  "eventPriority": "High",
  "eventType": "Event Driven",
  "eventLevel": "Log",
  "eventLogLevel": "info",
  "eventAudience": "Security",
  "eventAudienceTeam": "null",
  "eventFrequency": 0,
  "eventTags_0": "Threat Prevention",
  "eventTags_1": "Web Application & API Protection",
  "eventSource_agentId": "fbfb6a19-daf7-41e5-a25c-bf3a1472e97a",
  "eventSource_eventTraceId": "null",
  "eventSource_eventSpanId": "null",
  "eventSource_issuingEngineVersion": "1.1.26-open-source",
  "eventSource_serviceName": "HTTP Transaction Handler",
  "eventSource_serviceId": "1",
  "eventSource_assetId": "Any",
  "eventSource_assetName": "Any",
  "eventData_logIndex": 9,
  "eventData_eventReferenceId": "ca55b172-8ec9-4e32-a094-aa555da0b187",
  "eventData_assetId": "Any",
  "eventData_assetName": "Any",
  "eventData_eventConfidence": "Very High",
 

In [ ]:
print("\nLưu mô hình vào Google Drive...")
from google.colab import drive
drive.mount('/content/drive')
!cp -r /content/qwen3-4b-finetuned-security /content/drive/MyDrive/qwen3-4b-7-3
print("Mô hình đã được lưu vào /content/drive/MyDrive/qwen3-4b-finetuned-security-7-3")


Lưu mô hình vào Google Drive...
Mounted at /content/drive
Mô hình đã được lưu vào /content/drive/MyDrive/qwen3-4b-finetuned-security-7-3


In [ ]:
messages = [
      {"role": "user", "content": "What could go wrong if I write the code differently?"},

]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = True, # Disable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 1024, # Increase for longer outputs!
    temperature = 0.6, top_p = 0.95, top_k = 20, # For thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

```c
#include<stdio.h>
#include<stdlib.h>

int main() {
    int *a = (int*)malloc(10 * sizeof(int));
    if (a == NULL) {
        printf("Memory allocation failed.\n");
        return 1;
    }
    for (int i = 0; i < 10; i++) {
        scanf("%d", &a[i]);
    }
    printf("The sum is %d.\n", a[10]);
    free(a);
    return 0;
}
```
```c
#include<stdio.h>
#include<stdlib.h>

int main() {
    int *a = (int*)malloc(10 * sizeof(int));
    if (a == NULL) {
        printf("Memory allocation failed.\n");
        return 1;
    }
    for (int i = 0; i < 10; i++) {
        scanf("%d", &a[i]);
    }
    printf("The sum is %d.\n", a[10]);
    free(a);
    return 0;
}
```<|im_end|>


In [ ]:
import os

# Đường dẫn đến thư mục
output_dir = "/content/qwen3-4b-finetuned-security"

# Kiểm tra xem thư mục có tồn tại không
if os.path.exists(output_dir):
    print(f"Thư mục {output_dir} tồn tại. Nội dung:")
    # Liệt kê các thư mục con và tệp
    for item in os.listdir(output_dir):
        print(item)
        # Nếu là thư mục con (như checkpoint-10), liệt kê nội dung bên trong
        item_path = os.path.join(output_dir, item)
        if os.path.isdir(item_path):
            print(f"  Nội dung của {item}:")
            for sub_item in os.listdir(item_path):
                print(f"    {sub_item}")
else:
    print(f"Thư mục {output_dir} không tồn tại. Có thể huấn luyện chưa hoàn thành.")

Thư mục /content/qwen3-4b-finetuned-security tồn tại. Nội dung:
checkpoint-30
  Nội dung của checkpoint-30:
    scaler.pt
    vocab.json
    special_tokens_map.json
    merges.txt
    tokenizer_config.json
    scheduler.pt
    tokenizer.json
    optimizer.pt
    adapter_config.json
    trainer_state.json
    rng_state.pth
    README.md
    adapter_model.safetensors
    added_tokens.json
    chat_template.jinja
    training_args.bin
checkpoint-20
  Nội dung của checkpoint-20:
    scaler.pt
    vocab.json
    special_tokens_map.json
    merges.txt
    tokenizer_config.json
    scheduler.pt
    tokenizer.json
    optimizer.pt
    adapter_config.json
    trainer_state.json
    rng_state.pth
    README.md
    adapter_model.safetensors
    added_tokens.json
    chat_template.jinja
    training_args.bin
checkpoint-10
  Nội dung của checkpoint-10:
    scaler.pt
    vocab.json
    special_tokens_map.json
    merges.txt
    tokenizer_config.json
    scheduler.pt
    tokenizer.json
    optimizer.

In [ ]:
# Lưu LoRA adapters từ checkpoint cuối cùng
model.save_pretrained("./qwen3-4b-finetuned/checkpoint-30")  # Local saving
tokenizer.save_pretrained("./qwen3-4b-finetuned/checkpoint-30")
# model.push_to_hub("your_name/qwen3-4b-finetuned", token="...")  # Online saving
# tokenizer.push_to_hub("your_name/qwen3-4b-finetuned", token="...")  # Online saving

# Tải lại LoRA adapters để sử dụng (bật False thành True nếu cần)
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="./qwen3-4b-finetuned/checkpoint-30",  # Đường dẫn đến checkpoint
        max_seq_length=2048,
        load_in_4bit=True,
    )